In [21]:
!wget https://bakrianoo.s3-us-west-2.amazonaws.com/aravec/full_grams_sg_300_twitter.zip

--2021-01-16 19:55:03--  https://bakrianoo.s3-us-west-2.amazonaws.com/aravec/full_grams_sg_300_twitter.zip
Resolving bakrianoo.s3-us-west-2.amazonaws.com (bakrianoo.s3-us-west-2.amazonaws.com)... 52.218.205.49
Connecting to bakrianoo.s3-us-west-2.amazonaws.com (bakrianoo.s3-us-west-2.amazonaws.com)|52.218.205.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3322543144 (3.1G) [application/zip]
Saving to: ‘full_grams_sg_300_twitter.zip’

full_grams_sg_300_t 100%[===================>]   3.09G   874KB/s    in 52m 41s 

2021-01-16 20:47:46 (1.00 MB/s) - ‘full_grams_sg_300_twitter.zip’ saved [3322543144/3322543144]



In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [26]:
import pandas as pd
import re
import random
import string
import matplotlib.pyplot as plt
from utilities import clean_arabic_tweet
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.utils import to_categorical
from utilities import clean_arabic_tweet


import gensim
import numpy as np
from nltk import ngrams

In [3]:
pd.set_option('display.max_colwidth', -1)

## load train and test data

In [56]:
train_data_path = './cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = './cleaned_data/cleaned_test_data_for_subtask1.csv'

In [57]:
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [58]:
classes_names = train_data['#3_country_label'].unique().tolist()

In [7]:
classes_names

['Egypt',
 'Iraq',
 'Saudi_Arabia',
 'Mauritania',
 'Algeria',
 'Syria',
 'Oman',
 'Tunisia',
 'Lebanon',
 'Morocco',
 'Djibouti',
 'United_Arab_Emirates',
 'Kuwait',
 'Libya',
 'Bahrain',
 'Qatar',
 'Yemen',
 'Palestine',
 'Jordan',
 'Somalia',
 'Sudan']

In [8]:
classes_map={}
for i,class_name in enumerate(classes_names):
    classes_map[class_name]=i

In [9]:
classes_map

{'Egypt': 0,
 'Iraq': 1,
 'Saudi_Arabia': 2,
 'Mauritania': 3,
 'Algeria': 4,
 'Syria': 5,
 'Oman': 6,
 'Tunisia': 7,
 'Lebanon': 8,
 'Morocco': 9,
 'Djibouti': 10,
 'United_Arab_Emirates': 11,
 'Kuwait': 12,
 'Libya': 13,
 'Bahrain': 14,
 'Qatar': 15,
 'Yemen': 16,
 'Palestine': 17,
 'Jordan': 18,
 'Somalia': 19,
 'Sudan': 20}

In [62]:
def find_class_id_from_name(class_name):
    return classes_map[class_name]

In [63]:
train_data['classes_id']=train_data['#3_country_label'].apply(find_class_id_from_name)
test_data['classes_id']=train_data['#3_country_label'].apply(find_class_id_from_name)

In [64]:
train_data.head(10)

,#1_tweetid,#2_tweet,#3_country_label,#2_tweet_clean,classes_id
0,TRAIN_0,حاجة حلوة اكيد,Egypt,حاجه حلوه اكيد,0
1,TRAIN_1,عم بشتغلوا للشعب الاميركي اما نحن يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعدائهم,Iraq,عم بشتغلوا لشعب الاميركي اما نحن يكذبوا و يغشوا و يسرقوا و يقتلو شعوبهم و يعملوا لصالح اعداءهم,1
2,TRAIN_2,ابشر طال عمرك,Saudi_Arabia,ابشر طال عمرك,2
3,TRAIN_3,منطق 2017: أنا والغريب علي إبن عمي وأنا والغريب وإبن عمي علي أخويا. #قطع_العلاقات_مع_قطر #موريتانيا_مع_قطر,Mauritania,منطق انا و الغريب علي ابن عمي و انا و الغريب و ابن عمي علي اخويا قطع العلاقات مع قطر موريتانيا مع قطر,3
4,TRAIN_4,شهرين وتروح والباقي غير صيف ملينا,Algeria,شهرين و تروح و الباقي غير صيف ملينا,4
5,TRAIN_5,يابنتى والله ما حد متغاظ ولا مفروس منك ولا بيحسدك انتى عره اساسا.,Syria,يابنتي و اله ما حد متغاظ و لا مفروس منك و لا بيحسدك انتي عره اساسا,5
6,TRAIN_6,نفس الوقت بأكد على صاحبتي ان اي هدف هتحطه وتخططله هيبوظ والأفضل التشاؤم واننا نتوقع الأسوء دايما والفشل عشان منعشمش نفسنا ع الفاضي,Egypt,نفس الوقت باكد علي صاحبتي ان اي هدف هتحطه و تخطله هيبوظ و الافضل التشاءم و انا نتوقع الاسوء دايما و الفشل عشان منعشمش نفسنا ع الفاضي,0
7,TRAIN_7,م تبطلي خرا بقا علشان مطلعهوش عليكي احترمي نفسك URL …,Egypt,م تبطلي خرا بقا علشان مطلعهوش عليكي احترمي نفسك,0
8,TRAIN_8,ما يله دخل !,Oman,ما يله دخل,6
9,TRAIN_9,هو حلو بس يتخربط ع طلاب المدراس ليك مايغيرونه عدنا,Iraq,هو حلو بس يتخربط ع طلاب المدراس ليك مايغيرونه عدنا,1


In [65]:
test_data.head(10)

,#1_tweetid,#2_tweet,#3_country_label,#2_tweet_clean,classes_id
0,DEV_0,قولنا اون لاين لا يا علي اون لاين لا,Egypt,قولنا اون لاين لا ياعلي اون لاين لا,0
1,DEV_1,ههههه بايخه ههههه URL …,Oman,ه بايخه ه,1
2,DEV_2,ربنا يخليك يا دوك ولك المثل :D,Lebanon,ربنا يخليك يادوك و لك المثل,2
3,DEV_3,#اوامر_ملكيه ياشباب اي واحد فيكم عنده شي يذكره بالعساف يجيبه له احرقه,Syria,اوامر ملكيه ياشباب اي و احد فيكم عنده شي يذكره بالعساف يجيبه له احرقه,3
4,DEV_4,شد عالخط حتى هيا اكويسه,Libya,شد عالخط حتي هيا اكويسه,4
5,DEV_5,انا صاحي ميت مالضحك ايه داااااا,Egypt,انا صاحي ميت مالضحك ايه دا,5
6,DEV_6,لا مين أنتِ؟ ههههههههههههههههههههههههههههههههههههههههههههههههههههههه .,Saudi_Arabia,لا مين انت ه,0
7,DEV_7,جاي عبالي رشوف,Palestine,جاي عبالي رشوف,0
8,DEV_8,"""\""ان شاء الله تببوا إلي وإبقى إلكن كل العمر \"" أمينن يارب URL """,Morocco,ان شاء اله تبوا الي و ابقي الكن كل العمر امين يارب,6
9,DEV_9,الله يلعنك انت اصلا مريض انتزعت من قلبك الرحه URL …,Egypt,اله يلعنك انت اصلا مريض انتزعت من قلبك الرحه,1


In [66]:
train_x = train_data['#2_tweet_clean'].tolist()
train_y = train_data['classes_id'].tolist()

test_x = test_data['#2_tweet_clean'].tolist()
test_y = test_data['classes_id'].tolist()

In [67]:
train_x = [text_to_word_sequence(tweet) for tweet in train_x]
train_y = to_categorical(train_y)

test_x = [text_to_word_sequence(tweet) for tweet in test_x]
test_y = to_categorical(test_y)

## loading AraVec

In [24]:
t_model = gensim.models.Word2Vec.load('models/full_grams_sg_300_twitter.mdl')


In [39]:
# python 3.X
token1 = clean_arabic_tweet('عمرو')
token2 = clean_arabic_tweet('عمر')

if token in t_model.wv:
    most_similar = t_model.wv.most_similar( token2, topn=10 )
    for term, score in most_similar:
        term = clean_arabic_tweet(term).replace(" ", "_")
        if term != token:
            print(term, score)

وعمر 0.6840100288391113
عمر_ 0.6307801008224487
لعمر 0.6157344579696655
عمر 0.6152927875518799
عمرعمر 0.5917637348175049
ثاني_ 0.5843443870544434
عمر_ 0.5777392983436584
الوطيباني 0.57755446434021
عمر_ 0.5753136277198792
عندياطول 0.5683202743530273


In [70]:
word_vector = t_model.wv[ 'وطز' ]

In [71]:
word_vector.shape

(300,)

In [72]:
train_x_vectors, test_x_vectors = [] , []

In [74]:
OOV = []
for tweet in train_x:
    tweet_vector = []
    for token in tweet:
        try:
            tweet_vector.append(t_model.wv[ token ])
        except:
            OOV.append(token)
            
    train_x_vectors.append(tweet_vector)

In [77]:
OOV

['تخطله',
 'التشاءم',
 'منعشمش',
 'مطلعهوش',
 'هالنونا',
 'غيخلي',
 'غيلعبها',
 'هيتشرف',
 'عقلهاش',
 'لهالكلب',
 'الجزاءريه',
 'لاسطا',
 'تحبكي',
 'مايهزمني',
 'ظنيتش',
 'مءدن',
 'مايبزقك',
 'نخدعنابالنظاره',
 'صلهدعم',
 'جيهنا',
 'الكمزاري',
 'لجدانه',
 'افتيله',
 'لامراهقين',
 'لاذيول',
 'الرءاسي',
 'مابقاو',
 'يطيحلي',
 'المءت',
 'راهون',
 'لرجعيه',
 'ياايڤونا',
 'تناطل',
 'نقنعن',
 'بيهاحبيبتي',
 'روژايتي',
 'لشقاوتنا',
 'يتنحاو',
 'كاندوزو',
 'السطاج',
 'جاهنام',
 'حلطه',
 'سءال',
 'اكهاو',
 'حفرتني',
 'عضروطيه',
 'يارءي',
 'ياامباع',
 'بالقايم',
 'الحداڭه',
 'الجزايز',
 'كنلعب',
 'نصيبكربنا',
 'ياكاريو',
 'مونتاداسوءدادي',
 'عيشتا',
 'بيقروانحنا',
 'نتناقشفقط',
 'نتمنيينسمع',
 'يرادلج',
 'فكانسي',
 'بتوقعله',
 'فلونك',
 'الاندرغراوند',
 'لبرفورمنس',
 'يشاقه',
 'اجلنه',
 'كماج',
 'مانتسجلش',
 'لوسيلسو',
 'ندومبيلي',
 'ماشاءاله',
 'مهنشربش',
 'شيالف',
 'بالمصايح',
 'بنرفعك',
 '﮼ه',
 'متازه',
 'دانطيكم',
 'مخترعنه',
 'بالحديثكيف',
 'متحق',
 'نديرهلك',
 'لايكد',
 'ضغرا',
 'باربون',
